# Centros sociosanitarios y docentes

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dbfread import DBF
import geopandas as gpd
import datetime
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
centros = gpd.read_file('Data_input/Centros/Centros_Sociosanitarios_Docentes.shp')
centros.head()

,NOMBRE,TIPO,SUBTIPO,geometry
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (444737.000 4476780.000)
1,EL ENCUENTRO,CENTROS DE DÍA,HOGARES Y CLUBES,POINT (444238.000 4475696.000)
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (442684.000 4480112.000)
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES MIXTA,POINT (440154.000 4475830.000)
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (448266.000 4477239.000)


In [3]:
print('Número de centros sociosanitarios y docentes:', len(centros))

Número de centros sociosanitarios y docentes: 2258


In [4]:
centros.isnull().sum()

NOMBRE         0
TIPO           0
SUBTIPO     1816
geometry       0
dtype: int64

In [5]:
centros.nunique()

NOMBRE      2257
TIPO           5
SUBTIPO        7
geometry    2090
dtype: int64

In [6]:
centros['TIPO'].value_counts()

TIPO
CENTROS DOCENTES             1750
CENTROS DE DÍA                273
CENTROS RESIDENCIALES         168
CENTROS SANITARIOS             66
CENTROS DE REHABILITACIÓN       1
Name: count, dtype: int64

In [7]:
centros['SUBTIPO'].value_counts()

SUBTIPO
CENTROS DE DÍA: OTROS                           161
HOGARES Y CLUBES                                112
RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES    106
RESIDENCIAS DE PERSONAS MAYORES MIXTA            52
CENTROS RESIDENCIALES: OTROS                      6
RESIDENCIAS DE PERSONAS MAYORES AUTONOMAS         4
CENTROS DE ATENCIÓN                               1
Name: count, dtype: int64

La columna que tiene la mayoría de valores nulos es `SUBTIPO`. Se va a eliminar, ya que no es necesaria y el tipo ya es definido en otra columna.

In [8]:
centros.drop('SUBTIPO', inplace=True, axis=1)

In [9]:
centros.head()

,NOMBRE,TIPO,geometry
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,POINT (444737.000 4476780.000)
1,EL ENCUENTRO,CENTROS DE DÍA,POINT (444238.000 4475696.000)
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,POINT (442684.000 4480112.000)
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,POINT (440154.000 4475830.000)
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,POINT (448266.000 4477239.000)


El atributo `geometry` se transforma en longitud y latitud para ver si las coordenadas son válidas.

In [59]:
centros['LATITUD'] = centros.geometry.x
centros['LONGITUD'] = centros.geometry.y

In [38]:
centros.drop('geometry', axis=1, inplace=True)

In [ ]:
lon, lat = centro.iloc[i]['coord_x'], centro.iloc[i]['coord_y'], inverse=True
      centro.loc[i, 'latitud'] = lat
      centro.loc[i, 'longitud'] = lon

In [61]:
centros.loc[np.where(centros['LONGITUD']<4000000)]

,NOMBRE,TIPO,SUBTIPO,geometry,LATITUD,LONGITUD
132,CENTRO DE DIA FLORENCIA,CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
143,BALLESOL CENTRO RESIDENCIAL PASILLO VERDE,CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES MIXTA,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
159,"SANITAS RESIDENCIAL EL VISO, RESIDENCIA DE PER...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
201,"LAS VICTORIAS BEFEKO, CENTRO DE DIA",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
214,"ARTURO PAJUELO, CENTRO MUNICIPAL DE MAYORES",CENTROS DE DÍA,HOGARES Y CLUBES,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
275,"NUESTRA SEÑORA DE LOS ANGELES, CENTRO MUNICIPA...",CENTROS DE DÍA,HOGARES Y CLUBES,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
303,"JUAN PABLO II AYUNTAMIENTO DE MADRID, CENTRO D...",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
304,"PARQUE DE QUEROL, CENTRO DE DIA",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
306,"SANTA ENGRACIA, CENTRO DE DIA PARA PERSONAS MA...",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308
318,"MANUEL ALEXANDRE AYUNTAMIENTO DE MADRID, CENTR...",CENTROS DE DÍA,HOGARES Y CLUBES,POINT (-17976931348623157081452742373170435679...,-1.797693e+308,-1.797693e+308


In [71]:
geometries_errors = np.where(centros['LATITUD']<0)[0]
geometries_errors

array([ 132,  143,  159,  201,  214,  275,  303,  304,  306,  318,  345,
        357,  358,  373,  386,  397,  407,  427,  597, 2138], dtype=int64)

In [78]:
centros_correct = centros.drop(index=geometries_errors).reset_index(drop=True)
centros_correct

,NOMBRE,TIPO,SUBTIPO,geometry,LATITUD,LONGITUD
0,"NUEVA EDAD, CENTRO DE DIA",CENTROS DE DÍA,CENTROS DE DÍA: OTROS,POINT (444737.000 4476780.000),444737.0,4.476780e+06
1,EL ENCUENTRO,CENTROS DE DÍA,HOGARES Y CLUBES,POINT (444238.000 4475696.000),444238.0,4.475696e+06
2,"SANTA FILIPINA, RESIDENCIA DE PERSONAS MAYORES",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (442684.000 4480112.000),442684.0,4.480112e+06
3,"RSDCIA. SACERDOTAL SAN PEDRO, RESIDENCIA DE PE...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES MIXTA,POINT (440154.000 4475830.000),440154.0,4.475830e+06
4,"NTRA. SRA. DEL PILAR, RESIDENCIA DE PERSONAS M...",CENTROS RESIDENCIALES,RESIDENCIAS DE PERSONAS MAYORES DEPENDIENTES,POINT (448266.000 4477239.000),448266.0,4.477239e+06
...,...,...,...,...,...,...
2233,ESCUELA INFANTIL PEQUEANDO,CENTROS DOCENTES,None,POINT (444978.000 4482367.000),444978.0,4.482367e+06
2234,ESCUELA INFANTIL REGGIO,CENTROS DOCENTES,None,POINT (442867.000 4483980.000),442867.0,4.483980e+06
2235,ESCUELA INFANTIL TODO NIÑOS NURSERY SERRANO,CENTROS DOCENTES,None,POINT (442010.000 4477709.000),442010.0,4.477709e+06
2236,ESCUELA MUNICIPAL DE MÚSICA MORATALAZ,CENTROS DOCENTES,None,POINT (444196.000 4473679.000),444196.0,4.473679e+06


In [84]:
centros_correct2 = centros_correct.drop(['LATITUD', 'LONGITUD'], axis=1)

In [85]:
#centros_correct_clean = centros_correct2.to_file('Datasets/Centros/Centros_Sociosanitarios_Docentes_correct.shp', index=False)